# Doc2vecの文書ベクトルを使用し、scikit-learnでコサイン類似検索




- Wikipediaの文書からDoc2Vecモデルを生成


- scikit-learnのcosine-simularityを使用して類似検索


昨日検証時のモデルファイルをそのまま使用しています。

nosetestsの質問文を使用してテストしたところ、４問中１問だけが正解、という結果に終わりました。

## (1) Wikipediaコンテンツファイルから全文書を抽出

<a href="31-Wikipedia-contents-csv.ipynb"><b>こちらの手順</b></a> にて、いったんローカルPCにCSVファイル化しておきます。

## (2) Wikipedia文書を学習

<a href="23-Doc2Vec-with-Wiki.ipynb"><b>こちらの手順</b></a> にて生成したDoc2Vecモデルファイルをロードして使用します。

上記手順では、Wikipedia文書のみを使用し、ボキャブラリ／単語ベクトルの生成および学習を行い、モデルをファイル保存しています。

In [1]:
'''
    環境準備
'''
import sys
import os

import numpy as np
import pandas as pd
 
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)
if learning_dir not in sys.path:
    sys.path.append(learning_dir)

In [2]:
from gensim import models
from gensim.models.doc2vec import Doc2Vec

def doc2vec_model_path(dm):
    model_path = 'prototype/better_algorithm/doc2vec.wikipedia.PV%d.model' % dm
    return model_path

In [3]:
'''
    あらかじめ学習したモデルのファイルをロード
    dm = 0 : DBoWを使用したモデル
'''
dm = 0
loaded_model_dbow = models.Doc2Vec.load(doc2vec_model_path(dm))

print('Document vector size=%d' % (len(loaded_model_dbow.docvecs)))

Document vector size=87181


## (3) my-opeの文書を、Wikiから生成したモデルにより文書ベクトル化する関数

Wikipedia文書だけで学習されたDoc2Vecモデルを使用し、my-ope文書（質問文）をベクトル化します。

In [4]:
import numpy as np

from learning.core.learn.learning_parameter import LearningParameter
from learning.core.datasource import Datasource

_bot_id = 13 # toyotsu_human.csv
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': 0.5,
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    'params_for_algorithm': {'C': 140},
    'excluded_labels_for_fitting': None
}
learning_parameter = LearningParameter(attr)

In [5]:
import MeCab
import mojimoji

class Nlang_naive:
    @classmethod
    def split(self, text):
        tagger = MeCab.Tagger("-u learning/dict/custom.dic")
        tagger.parse('')  # node.surfaceを取得出来るようにするため、空文字をparseする(Python3のバグの模様)
        node = tagger.parseToNode(text)
        word_list = []
        while node:
            features = node.feature.split(",")
            pos = features[0]
            if pos in ["BOS/EOS", "記号"]:
                node = node.next
                continue

            #print(features)
            lemma = node.feature.split(",")[6]

            if lemma == "*":
                lemma = node.surface  #.decode("utf-8")
                
            word_list.append(mojimoji.han_to_zen(lemma))
            node = node.next
        return " ".join(word_list)

    @classmethod
    def batch_split(self, texts):
        splited_texts = []
        for text in texts:
            splited_texts.append(self.split(text))
        return splited_texts

In [6]:
def get_document_vector(question, model):
    '''
        question: 
            分かち書きされていない文書
        model:
            Doc2Vecの学習済みモデル
            （検証時は品詞を落としていないWikipedia文書からモデルを生成）

        inferred_vector:
            文書を分かち書きしたコーパスから、
            Doc2Vecの学習済みモデルを使用して
            生成される類似文書ベクトル
            （モデルに合わせ、品詞は落とさない様にする）
    '''
    corpus = Nlang_naive.split(question).split()
    inferred_vector = model.infer_vector(corpus)

    return inferred_vector

def get_document_vectors(questions, model):
    document_vectors = []
    for question in questions:
        inferred_vector = get_document_vector(question, model)
        document_vectors.append(list(inferred_vector))

    return np.array(document_vectors)

## (4) コサイン類似検索の実行

質問文は、my-ope プロダクションの nosetests テストケースから引用しました。

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
from learning.core.datasource import Datasource

def search_simiarity(question, dbow_model):
    '''
        質問文間でコサイン類似度を算出して、近い質問文の候補を取得する
        
        仕様はプロダクションに準拠しています
        ただし、文書のベクトル化は、TF-IDFではなく、
        Doc2Vecを使用します。
    '''
    datasource = Datasource('csv')
    question_answers = datasource.question_answers_for_suggest(_bot_id, question)

    #all_array = TextArray(question_answers['question'], vectorizer=self.vectorizer)
    #question_array = TextArray([question], vectorizer=self.vectorizer)
    all_array      = get_document_vectors(question_answers['question'], dbow_model)
    question_array = get_document_vectors([question], dbow_model)
    
    print('count: my-ope all questions=%d, document vectors=%d (features=%d)' % (
        np.size(question_answers['question']), all_array.shape[0], all_array.shape[1]
    ))    
    print('count: question=%d, document vectors=%d (features=%d)' % (
        np.size([question]), question_array.shape[0], question_array.shape[1]
    ))    

    similarities = cosine_similarity(all_array, question_array)
    similarities = similarities.flatten()

    ordered_result = list(map(lambda x: {
        'question_answer_id': float(x[0]), 'similarity': x[1], 'answer_id': x[2]
    }, sorted(zip(question_answers['id'], similarities, question_answers['answer_id']), key=lambda x: x[1], reverse=True)))

    df = pd.DataFrame.from_dict(ordered_result)

    print(df[0:10])

In [8]:
# 正解＝6803
search_simiarity('JAL マイレージ', loaded_model_dbow)

2017/05/29 PM 03:44:01 ['./fixtures/learning_training_messages/benefitone.csv', './fixtures/learning_training_messages/ptna.csv', './fixtures/learning_training_messages/septeni.csv', './fixtures/learning_training_messages/toyotsu_human.csv']
2017/05/29 PM 03:44:01 ['./fixtures/question_answers/toyotsu_human.csv']


count: my-ope all questions=317, document vectors=317 (features=200)
count: question=1, document vectors=1 (features=200)
   answer_id  question_answer_id  similarity
0       6819             13393.0    0.195777
1       6823             13397.0    0.152361
2       6802             13377.0    0.141884
3       6911             13485.0    0.140777
4       6745             13317.0    0.135494
5       6860             13434.0    0.127516
6       6968             13548.0    0.126695
7       6986             13566.0    0.125811
8       6834             13408.0    0.120590
9       6966             13546.0    0.115356


In [9]:
# 正解＝6763
search_simiarity('海外の出張費の精算の方法は？', loaded_model_dbow)

2017/05/29 PM 03:44:02 ['./fixtures/learning_training_messages/benefitone.csv', './fixtures/learning_training_messages/ptna.csv', './fixtures/learning_training_messages/septeni.csv', './fixtures/learning_training_messages/toyotsu_human.csv']
2017/05/29 PM 03:44:02 ['./fixtures/question_answers/toyotsu_human.csv']


count: my-ope all questions=317, document vectors=317 (features=200)
count: question=1, document vectors=1 (features=200)
   answer_id  question_answer_id  similarity
0       6749             13310.0    0.506196
1       6759             13331.0    0.479292
2       6766             13338.0    0.464680
3       6743             13314.0    0.441656
4       6856             13430.0    0.432572
5       6744             13316.0    0.414209
6       6863             13437.0    0.411949
7       6763             13335.0    0.410406
8       6851             13425.0    0.409212
9       6792             13367.0    0.398344


In [10]:
# 正解＝6767
search_simiarity('VISAの勘定科目がわからない', loaded_model_dbow) 

2017/05/29 PM 03:44:02 ['./fixtures/learning_training_messages/benefitone.csv', './fixtures/learning_training_messages/ptna.csv', './fixtures/learning_training_messages/septeni.csv', './fixtures/learning_training_messages/toyotsu_human.csv']
2017/05/29 PM 03:44:02 ['./fixtures/question_answers/toyotsu_human.csv']


count: my-ope all questions=317, document vectors=317 (features=200)
count: question=1, document vectors=1 (features=200)
   answer_id  question_answer_id  similarity
0       6710             13280.0    0.350886
1       6916             13495.0    0.350341
2       6775             13348.0    0.327487
3       6933             13512.0    0.327060
4       6917             13496.0    0.325855
5       6807             13382.0    0.323945
6       6859             13433.0    0.323725
7       6787             13360.0    0.322733
8       6907             13481.0    0.322318
9       6804             13379.0    0.321103


In [11]:
# 正解＝6909
search_simiarity('子供が生まれた', loaded_model_dbow) 

2017/05/29 PM 03:44:02 ['./fixtures/learning_training_messages/benefitone.csv', './fixtures/learning_training_messages/ptna.csv', './fixtures/learning_training_messages/septeni.csv', './fixtures/learning_training_messages/toyotsu_human.csv']
2017/05/29 PM 03:44:03 ['./fixtures/question_answers/toyotsu_human.csv']


count: my-ope all questions=317, document vectors=317 (features=200)
count: question=1, document vectors=1 (features=200)
   answer_id  question_answer_id  similarity
0       6909             13483.0    0.413178
1       6985             13565.0    0.287570
2       6871             13445.0    0.282850
3       6853             13427.0    0.274783
4       6773             13345.0    0.271242
5       6807             13382.0    0.270986
6       6924             13503.0    0.269384
7       6986             13566.0    0.268611
8       6738             13324.0    0.260625
9       6888             13462.0    0.260544
